## Pandas Exercise

### Part 1: Aggregation, Group By and Sort Review

Below is a dataframe, with values as a list-of-lists and columns as a list.

In [2]:
import pandas as pd

df = pd.DataFrame(
    [[123, 'xt23', 20], [123, 'q45', 2], [123, 'a89', 25], [77, 'q45', 3],
     [77, 'a89', 30], [92, 'xt23', 24], [92, 'm33', 60], [92, 'a89', 28]],
    columns=['userid', 'product', 'price'])
df

,userid,product,price
0,123,xt23,20
1,123,q45,2
2,123,a89,25
3,77,q45,3
4,77,a89,30
5,92,xt23,24
6,92,m33,60
7,92,a89,28


**Q: We want the maximum price anyone has paid.**

In [3]:
df['price'].max()

60

In [4]:
df[df['price'] == df['price'].max()]

,userid,product,price
6,92,m33,60


**Q: We want the maximum price per user. Hint: Use groupby.**

In [5]:
df.groupby('userid')[['price']].max()

,price
userid,
77,30
92,60
123,25


**Q: What is the total amount paid by user?**

Note: Pandas will smartly leave out columns for which that aggregation doesn't have meaning.

In [6]:
df.groupby('userid')[['price']].sum()

,price
userid,
77,33
92,112
123,47


**Q: Sort the dataframe by userid first, and then price, both ascending.**

In [7]:
df.sort_values(['userid', 'price'], ascending=[True, True])

,userid,product,price
3,77,q45,3
4,77,a89,30
5,92,xt23,24
7,92,a89,28
6,92,m33,60
1,123,q45,2
0,123,xt23,20
2,123,a89,25


### Part 2: Combining Techniques

**Q: We want the maximum price each user paid, and the product associated with that price.**

Note for SQL users: In SQL, you groupby and then sort, but in pandas, it's easier to do it the other way around.

In [8]:
(df.sort_values(['userid', 'price'], ascending=[True, False]).groupby('userid')
 .head(1))

,userid,product,price
4,77,a89,30
6,92,m33,60
2,123,a89,25


### Part 3: Exploring The Index

Let's add a new column.

In [9]:
df['website'] = [
    'Amazon', 'Amazon', 'NewEgg', 'NewEgg', 'NewEgg', 'Amazon', 'Amazon',
    'Amazon'
]
df

,userid,product,price,website
0,123,xt23,20,Amazon
1,123,q45,2,Amazon
2,123,a89,25,NewEgg
3,77,q45,3,NewEgg
4,77,a89,30,NewEgg
5,92,xt23,24,Amazon
6,92,m33,60,Amazon
7,92,a89,28,Amazon


**Q: What is the total amount paid by each user on each website?**

In [10]:
df.groupby(['userid', 'website']).sum()

price
userid website       
77     NewEgg      33
92     Amazon     112
123    Amazon      22
       NewEgg      25

Now let's do the same groupby as above, but set the as_index flag to "False". This will result in a flat table instead of the nested indexes.

In [11]:
df.groupby(['userid', 'website'], as_index=False).sum()

,userid,website,price
0,77,NewEgg,33
1,92,Amazon,112
2,123,Amazon,22
3,123,NewEgg,25


### Part 4: Merging

Let's now create a second table:

In [12]:
df2 = pd.DataFrame(
    [[123, 'USA'], [77, 'Canada'], [92, 'USA']], columns=['userid', 'country'])
df2

,userid,country
0,123,USA
1,77,Canada
2,92,USA


We can combine the two tables using a merge function. What it does is, it will do a pairwise comparision of every row in table1 with every row in table2 and if the "on" condition matches, it will create a single row with columns from both those matched rows.

Merge of two tables with 5 rows each can give as little as 0 rows and as much as 25 rows.

    [1,2,3,4,5] merged with [6,7,8,9,10] will give 0 rows
    [1,2,3,4,5] merged with [1,2,3,4,5] will give 5 rows
    [1,1,1,1,1] merged with [1,1,1,1,1] will give 25 rows

In [13]:
pd.merge(df, df2, on='userid')

,userid,product,price,website,country
0,123,xt23,20,Amazon,USA
1,123,q45,2,Amazon,USA
2,123,a89,25,NewEgg,USA
3,77,q45,3,NewEgg,Canada
4,77,a89,30,NewEgg,Canada
5,92,xt23,24,Amazon,USA
6,92,m33,60,Amazon,USA
7,92,a89,28,Amazon,USA


**Q: What is the total amount paid per country?**

In [15]:
pd.merge(df, df2, on='userid').groupby(['country'])[['price']].sum()

,price
country,
Canada,33
USA,159


**Q: What is the average amount paid per country and website?**

In [14]:
(pd.merge(df, df2, on='userid').groupby(['country', 'website'])[['price']]
 .mean())

price
country website       
Canada  NewEgg    16.5
USA     Amazon    26.8
        NewEgg    25.0

### Part 5: The Final Question - A Demo

Let's add another column: purchase date

In [16]:
df['date'] = [
    '2018-01-12', '2018-01-08', '2018-01-06', '2018-01-03', '2018-01-05',
    '2018-01-04', '2018-01-07', '2018-01-02'
]
df

,userid,product,price,website,date
0,123,xt23,20,Amazon,2018-01-12
1,123,q45,2,Amazon,2018-01-08
2,123,a89,25,NewEgg,2018-01-06
3,77,q45,3,NewEgg,2018-01-03
4,77,a89,30,NewEgg,2018-01-05
5,92,xt23,24,Amazon,2018-01-04
6,92,m33,60,Amazon,2018-01-07
7,92,a89,28,Amazon,2018-01-02


**Q: Here is a tricky task. For each row, I want the average purchase price for that user prior to that purchase.**

One option is to do some loops. But another solution is to just do a merge on itself and filter.

### Movie / Wine Example

But first, quick question:

Say you merge (or 'join' if you come from SQL) two dataframes with 3 rows each, how many rows would you end up with?

Could be anything between 0-9.

Consider the following examples, where table x has users and the movies they like. And table y has users and the wines they line. And let's do a merge to come up with possible movie and wine pairings for any user. In case A, we get 0 rows, in case B, we get 3 rows and case C we get 9 rows.

#### Merge Two Tables with No IDs in Common

In [ ]:
dfx = pd.DataFrame(
    [[1, 'Godfather'], [2, 'Amelie'], [3, 'Chicago']],
    columns=['userid', 'movies'])
dfx

In [ ]:
dfy = pd.DataFrame(
    [[4, 'red'], [5, 'white'], [6, 'pink']], columns=['userid', 'wines'])
dfy

In [ ]:
dfm = pd.merge(dfx, dfy, on='userid')
dfm

#### Merge Two Tables with IDs in Common (1 Value Per ID)

In [ ]:
dfx = pd.DataFrame(
    [[1, 'Godfather'], [2, 'Amelie'], [3, 'Chicago']],
    columns=['userid', 'movies'])
dfx

In [ ]:
dfy = pd.DataFrame(
    [[1, 'red'], [2, 'white'], [3, 'pink']], columns=['userid', 'wines'])
dfy

In [ ]:
dfm = pd.merge(dfx, dfy, on='userid')
dfm

#### Merge Two Tables with IDs in Common (Multiple Values Per ID)

In [ ]:
dfx = pd.DataFrame(
    [[1, 'Godfather'], [1, 'Amelie'], [1, 'Chicago']],
    columns=['userid', 'movies'])
dfx

In [ ]:
dfy = pd.DataFrame(
    [[1, 'red'], [1, 'white'], [1, 'pink']], columns=['userid', 'wines'])
dfy

In [ ]:
dfm = pd.merge(dfx, dfy, on='userid')
dfm

### Back to the Question

Now let's return to the original question: For each row, I want the average purchase price for that user prior to that purchase. Let's do a merge on itself and filter.

Here are the steps we're going to take:
1. **MERGE**: Join table on itself. For each userid / date combo, show me all userid / date / price combos.
1. **FILTER**: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.
1. **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.
1. **MERGE**: Combine these values with the original dataframe for the final result.
1. **SORT**: Sort to make the results look pretty.

#### MERGE: Join table on itself. For each userid / date combo, show me all userid / date / price combos.

In [ ]:
df_date = df[['userid', 'date']]
df_date

In [ ]:
df_all = df[['userid', 'price', 'date']]
df_all

In [ ]:
df2 = pd.merge(df_date, df_all, on='userid')
df2

#### FILTER: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.

In [ ]:
df3 = df2[df2['date_x'] > df2['date_y']]
df3

#### **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.

In [ ]:
df4 = df3.groupby(['userid', 'date_x'])[['price']].mean()
df4.rename(columns={'price': 'avg_prior_price'}, inplace=True)
df4

#### **MERGE**: Combine these values with the original dataframe for the final result.

In [ ]:
df

In [ ]:
df4.index

In [ ]:
final = df.merge(
    df4,
    left_on=['userid', 'date'],
    right_on=['userid', 'date_x'],
    right_index=True,
    how='left')
final

#### **SORT**: Sort to make the results look pretty.

In [ ]:
final.sort_values(by=['userid', 'date'])